In [3]:
import pandas as pd 
import numpy as np 
from scipy.stats.mstats            import winsorize
from pandas                        import get_dummies
from sklearn.linear_model          import LogisticRegression
from sklearn.tree                  import DecisionTreeClassifier
from sklearn.ensemble              import RandomForestClassifier
from sklearn.ensemble              import GradientBoostingClassifier
from sklearn.svm                   import SVC
from sklearn.neural_network        import MLPClassifier
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.metrics               import accuracy_score
from sklearn.metrics               import auc
from sklearn.metrics               import roc_auc_score
from scipy.stats.mstats            import winsorize
from scipy.stats                   import pearsonr
from sklearn.linear_model          import LinearRegression
from sklearn.neighbors             import KNeighborsRegressor
from sklearn.ensemble              import GradientBoostingRegressor
from pandas                        import DataFrame
from sklearn.metrics               import mean_squared_error
from sklearn.model_selection       import cross_validate
from sklearn.ensemble              import RandomForestRegressor
from sklearn.model_selection       import train_test_split
from sklearn.linear_model          import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn                       import linear_model
from sklearn.ensemble              import BaggingClassifier
from sklearn.naive_bayes           import GaussianNB
from sklearn.ensemble              import StackingClassifier
from sklearn.preprocessing         import KBinsDiscretizer
from sklearn.model_selection       import GridSearchCV
from sklearn.ensemble              import AdaBoostClassifier
from sklearn.svm                   import SVC
from sklearn.pipeline              import make_pipeline
from sklearn.preprocessing         import StandardScaler
from sklearn.calibration           import CalibratedClassifierCV
from sklearn.svm                   import LinearSVC
from sklearn.model_selection       import cross_val_score
from sklearn.ensemble              import StackingClassifier

In [4]:
# Import Data 
train = pd.read_csv('orange_churn_train.csv')
test = pd.read_csv('orange_churn_test.csv')

In [5]:
train.head()

,cust_id,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,churn
0,3,NaN,NaN,NaN,NaN,NaN,1351.0,7.0,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,ELof,7P5s,ZI9m,NoEd,mj86,NaN,0
1,4,NaN,NaN,NaN,NaN,NaN,644.0,0.0,NaN,NaN,...,76DJixu,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN,0
2,7,NaN,NaN,NaN,NaN,NaN,2583.0,0.0,NaN,NaN,...,I5dzv5f,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,1
3,8,NaN,NaN,NaN,NaN,NaN,1463.0,7.0,NaN,NaN,...,xwyAw04,LM8l689qOp,NaN,kG3k,fKCe,RAYp,F2FyR07IdsN7I,mj86,NaN,0
4,9,NaN,NaN,NaN,NaN,NaN,77.0,0.0,NaN,NaN,...,76DJixu,NaN,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN,0


# DATA PROCESSING TRAIN

In [6]:
# Recap before cleaning
print( "col =", len(train.iloc[:, 1:-1].columns))
print("NaN=", train.isna().sum().sum())

col = 230
NaN= 1604602


## 1) Drop column with only NaN

In [7]:
# Number of columns with only NaN
train.isna().sum().to_list().count(10000)
# Delete the columns which have only na
train = train.dropna(axis=1, how='all')

print("Col =", len(train.columns))
print("NaN=", train.isna().sum().sum())

Col = 214
NaN= 1424602


## 2) Flag on NaN numeric

In [8]:
# Create a NaN dummy columns for each numerical column (as a NaN Flag)
numeric_nan_flag = pd.get_dummies(train._get_numeric_data().iloc[:, 1:-1].astype(str).replace('nan', np.nan), dummy_na=True).filter(regex='nan')
numeric_nan_flag.head()

,Var1_nan,Var2_nan,Var3_nan,Var4_nan,Var5_nan,Var6_nan,Var7_nan,Var9_nan,Var10_nan,Var11_nan,...,Var180_nan,Var181_nan,Var182_nan,Var183_nan,Var184_nan,Var186_nan,Var187_nan,Var188_nan,Var189_nan,Var190_nan
0,1,1,1,1,1,0,0,1,1,1,...,1,0,1,1,1,1,1,1,1,1
1,1,1,1,1,1,0,0,1,1,1,...,1,0,1,1,1,1,1,1,1,1
2,1,1,1,1,1,0,0,1,1,1,...,1,0,1,1,1,1,1,1,0,1
3,1,1,1,1,1,0,0,1,1,1,...,1,0,1,1,1,1,1,1,1,1
4,1,1,1,1,1,0,0,1,1,1,...,1,0,1,1,1,1,1,1,1,1


## 3) Numerical 

### a. Thresh 0.7 + fillna(mean)

In [9]:
# On the columns with less than 0.7 NaN, fillna with the mean 
numeric_train = train._get_numeric_data().dropna(thresh=0.7*len(train), axis=1).iloc[:, 1:-1]

for i in numeric_train.columns:
    numeric_train[i] = numeric_train[i].fillna(numeric_train[i].mean())

In [10]:
# Selection of only 39 columns (cut 0.7)
print("NA = ",numeric_train.isna().sum().sum())
print("Col =", len(numeric_train.columns))

NA =  0
Col = 39


### b. Bellow the thresh 0.7 -> cut + get_dummies

In [11]:
# Select all the column deleted previously with the cut 0.7 
diff = (set(numeric_train.columns)).symmetric_difference(set(train._get_numeric_data()))
diff = list(diff)
diff.pop(1)
print("Columns droped with the thresh:", len(diff))

Columns droped with the thresh: 136


In [12]:
# Create get dummies on numerical value 
# 1. For each columns, cut number depending on the number of unique values 
# 2. With cut create categorical data 
# 3. Used the data transformed in dummies variables 
df_cut = pd.DataFrame()

for i in diff:
    if len(pd.unique(train[i])) < 5:
        group = 4
    elif (len(pd.unique(train[i])) > 5) & (len(pd.unique(train[i])) < 12):
        group = 7
    else: 
        group = 10
    df_cut[i] = list(pd.cut(train[i], group).astype('str').replace('nan',np.nan))

    
df_cut = pd.get_dummies(df_cut)

In [13]:
df_cut.head()

,"Var104_(-0.621, 62.1]","Var104_(124.2, 186.3]","Var104_(186.3, 248.4]","Var104_(248.4, 310.5]","Var104_(310.5, 372.6]","Var104_(372.6, 434.7]","Var104_(434.7, 496.8]","Var104_(496.8, 558.9]","Var104_(558.9, 621.0]","Var104_(62.1, 124.2]",...,"Var145_(2592.0, 2880.0]","Var145_(288.0, 576.0]","Var145_(576.0, 864.0]","Var23_(-0.115, 11.5]","Var23_(103.5, 115.0]","Var23_(11.5, 23.0]","Var23_(23.0, 34.5]","Var23_(34.5, 46.0]","Var23_(46.0, 57.5]","Var23_(57.5, 69.0]"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Register the cut for each columns to be reused for the testset
cut_array = {}

for i in diff:  
    cut_array[i] = pd.cut(train[i], group, retbins=True)[1]

## 4) Characterical 

### a. Keep in characterical for trees algorithme

In [15]:
# Keep characterical values for the decision tree algo 

# Replace NaN with missing (but it is optional) at leat in str 
characteritic_train = train.select_dtypes(include='object').replace(np.nan, "Missing")

print("NA = ",characteritic_train.isna().sum().sum())
print("Col =", len(characteritic_train.columns))

NA =  0
Col = 38


### b. Get dummies 

In [16]:
#With characteristic values create dummies
characteritic_train_dummies = pd.get_dummies(characteritic_train)

print("NA = ",characteritic_train_dummies.isna().sum().sum())
print("Col =", len(characteritic_train_dummies.columns))

NA =  0
Col = 27572


## 5) Merge

In [17]:
# Add the customer_id columns
db = pd.DataFrame(train["cust_id"]).join(numeric_nan_flag)
db = db.join(numeric_train)

#DATABASE 1 - with charactere for trees models churn
db_charactere = db.join(characteritic_train)
db_charactere = db_charactere.join(pd.DataFrame(train["churn"]))  #Add churn column

# DATABASE 2 - only numeric variables for all models 
db_numeric = db.join(characteritic_train_dummies)
db_numeric = db_numeric.join(df_cut)
db_numeric = db_numeric.join(pd.DataFrame(train["churn"])) #Add churn column

In [18]:
# Verification/Recap 
print("Na =",db.isna().sum().sum(), "col=", len(db.columns), len(db))
print("Na =",db_charactere.isna().sum().sum(), "col=", len(db_charactere.columns), len(db_charactere))
print("Na =",db_numeric.isna().sum().sum(), "col=", len(db_numeric.columns), len(db_numeric))

Na = 0 col= 214 10000
Na = 0 col= 253 10000
Na = 0 col= 28625 10000


# DATA PROCESSING TEST SET 

In [19]:
#1) Drop column with only NaN
test = test.dropna(axis=1, how='all')
# 2) Flag on NaN numeric
test_numeric_nan_flag = pd.get_dummies(test._get_numeric_data().iloc[:, 1:-1].astype(str).replace('nan', np.nan), dummy_na=True).filter(regex='nan')
## 3) Numerical - thresh 0.7 + fillna(mean)
numeric_test = test._get_numeric_data().dropna(thresh=0.7*len(test), axis=1).iloc[:, 1:-1]

## 3a.
for i in numeric_test.columns:
    numeric_test[i] = numeric_test[i].fillna(numeric_test[i].mean())
## 4b.    
diff_test = (set(numeric_test.columns)).symmetric_difference(set(test._get_numeric_data()))
diff_test = list(diff_test)

common_var = []

for i in diff:
    if i in diff_test:
        common_var.append(i)

df_cut_test = pd.DataFrame()

for i in common_var:
    df_cut_test[i] = list(pd.cut(test[common_var][i], cut_array[i]).astype('str').replace('nan',np.nan))

    
df_cut_test = pd.get_dummies(df_cut_test)
    
# 4b. Get dummies 
characteritic_test = test.select_dtypes(include='object').replace(np.nan, "Missing")
characteritic_test_dummies = pd.get_dummies(characteritic_test)

# Merge 
test = pd.DataFrame(test["cust_id"]).join(test_numeric_nan_flag)
test = test.join(numeric_test)
test = test.join(df_cut_test)
test = test.join(characteritic_test_dummies)

print("Na =",test.isna().sum().sum(), "col=", len(test.columns), "row:", len(test))

Na = 0 col= 28516 row: 10000


## Common columns in the Train and Test

In [20]:
# On more than 27000 columns, half of them are in commun (because of the charaterical dummies mainly)
common_col = []

for i in db_numeric.columns.to_list():
    if i in test.columns.to_list():
        common_col.append(i)

In [21]:
# Number of common columns 
len(common_col)

13483

## Standardization 

In [22]:
#from sklearn import preprocessing
#scaler = preprocessing.StandardScaler().fit(db_numeric[common_col])
#train_common_col_standardize = scaler.transform(db_numeric[common_col])

In [23]:
#scaler = preprocessing.StandardScaler().fit(test[common_col])
#test_common_col_standardize = scaler.transform(test[common_col])

In [24]:
#train_common_col_standardize = pd.DataFrame(train_common_col_standardize)
#test_common_col_standardize = pd.DataFrame(test_common_col_standardize)

## PCA 

In [25]:
#from sklearn.decomposition import PCA

#pca = PCA(n_components=400)
#train_common_col_standardize_PCA = pca.fit_transform(train_common_col_standardize)
#test_common_col_standardize_PCA = pca.transform(test_common_col_standardize)

In [26]:
#train_common_col_standardize_PCA = pd.DataFrame(train_common_col_standardize_PCA)
#test_common_col_standardize_PCA = pd.DataFrame(test_common_col_standardize_PCA)

## PLS

In [27]:
#from sklearn.cross_decomposition import PLSRegression

#pls2 = PLSRegression()
#train_common_col_standardize_PLS = pls2.fit_transform(train_common_col_standardize)
#test_common_col_standardize_PLS = pls2.transform(test_common_col_standardize)

#train_common_col_standardize_PLS = pd.DataFrame(train_common_col_standardize_PCA)
#test_common_col_standardize_PLS = pd.DataFrame(test_common_col_standardize_PCA)

# MODEL

### Train model & Performance

In [28]:
## ON DB_numeric  (only numeric var without characterical var)
# Verification len matching
print(len(db_numeric[common_col].columns))
print(len(test[common_col].columns))

13483
13483


### Feature selection 

In [45]:
data = db_numeric[common_col].join(train["churn"]) 

In [46]:
# Loop Run many times the 2 following cell 
# To fine-tuning the number of features 

In [59]:
# Run first and then put as #comment
data = db_numeric[common_col].join(train["churn"]) 
# Run second once the personr run once 
###### data = db_numeric[common_col][selectedFeatures].join(train["churn"]) 
trainingSet, testSet = train_test_split(data, test_size=0.3)

In [60]:
trainingSet_features = trainingSet.columns[0:-1]
testSet_features = testSet.columns[0:-1] 

target = 'churn'

In [61]:
selectedFeatures = []
target = 'churn'

p_val = 0.05

trainingSet_columns = trainingSet_features

for column in trainingSet_columns:
        (pearson,pvalue) = pearsonr(trainingSet[column],trainingSet[target])
        (f"{column} - p-value = {pvalue} - selected : {1 if pvalue < p_val else 0}")
        if pvalue < p_val:
            selectedFeatures.append(column)
            
print("Selected features:",round(len(selectedFeatures)/len(trainingSet_features),2),"%")
print("Number of features:", len(selectedFeatures))

Selected features: 1.0 %
Number of features: 131


In [34]:
## Combine with cross validation to see the mean score of with the new features number 

In [63]:
# Models
tree          = DecisionTreeClassifier()
logistic      = LogisticRegression(solver = "lbfgs", max_iter = 500)
randomForest  = RandomForestClassifier()
boostedTree   = GradientBoostingClassifier()
boostedTree1  = GradientBoostingClassifier()
neuralNet     = MLPClassifier()
neighbors     = KNeighborsClassifier()
SGDClassifier = linear_model.SGDClassifier(loss="log")
lda           = LinearDiscriminantAnalysis()
svc           = make_pipeline(StandardScaler(), CalibratedClassifierCV(LinearSVC()))
gnb           = GaussianNB()
stackClass    = StackingClassifier(estimators=[(LinearDiscriminantAnalysis())], final_estimator=GradientBoostingClassifier())
bagging       = BaggingClassifier(base_estimator=GradientBoostingClassifier(), n_jobs=-1)
bagging2      = BaggingClassifier(base_estimator=GradientBoostingClassifier(), n_jobs=-1)
bagging1      = make_pipeline(StandardScaler(), BaggingClassifier(base_estimator=CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
AdaBoost      = AdaBoostClassifier()


models = {#"tree"         :tree,
          #"logistic"     :logistic,
          "randomForest" :randomForest,
          "boostedTree"  :boostedTree,
          #"boostedTree1" :boostedTree1,
          #"svc"          :svc,
          #"neuralNet"    :neuralNet,
          #"neighbors"    :neighbors,
          #"sgdc"         :SGDClassifier,
          "lda"          :lda,
          #"gnb"          :gnb,
          "bagging"       :bagging,
          "bagging2"      :bagging2,
          #"AdaBoost"     :AdaBoost
         }

In [64]:
for model in models:
    models[model].fit(trainingSet[selectedFeatures],trainingSet[target])
    print(f"{model} has been trained successfully")

randomForest has been trained successfully
boostedTree has been trained successfully
lda has been trained successfully
bagging has been trained successfully
bagging2 has been trained successfully


In [65]:
performances = {}

for model in models:
    #predictions   = models[model].predict(testSet[testSet_features])
    #probabilities = DataFrame(models[model].predict_proba(testSet[testSet_features]))[1]
    
    predictions   = models[model].predict(testSet[selectedFeatures])
    probabilities = DataFrame(models[model].predict_proba(testSet[selectedFeatures]))[1]
    accuracy      = accuracy_score(testSet[target],predictions)
    auc           = roc_auc_score(np.array(testSet[target]),np.array(probabilities))
    
    performances[model] = {"Accuracy":accuracy,"AUC":auc}

pd.DataFrame(performances)

,randomForest,boostedTree,lda,bagging,bagging2
Accuracy,0.924333,0.923667,0.920667,0.924667,0.925000
AUC,0.670454,0.737474,0.714794,0.746038,0.737006


##### Cross Validation - Confirm the observations

In [868]:
cross_val_score(bagging, db_numeric[common_col][variables_132],data[target],scoring="roc_auc", cv=20).mean()

0.7386735217403484

In [ ]:
#cross_val_score(boostedTree, db_numeric[common_col][variables_132],data[target],scoring="roc_auc", cv=20).mean()

#### Grid Search - Hyperparameters

In [901]:
#bagging    = BaggingClassifier(base_estimator=GradientBoostingClassifier(), n_jobs=-1)
#parameters = {'n_estimators' : [12], 'max_samples': [0.55, 0.6, 0.65, 0.7]}

#GridSearch = GridSearchCV(bagging, parameters, scoring = 'roc_auc',cv=10, n_jobs=-1)
#GridSearch.fit(data[selectedFeatures], data[target])
#GridSearch.best_params_

In [57]:
# Best feature selection 132 variables 

variables_132 = ['Var126_nan', 'Var7','Var13','Var65','Var73','Var81','Var113','Var126','Var134',
                         'Var140','Var144','Var192_1KSqrQK2Mx','Var192_AhzZptzip9','Var192_J9Vr4RQZiT',
                         'Var192_YddrjIQ5Ph','Var192_vAsvyx7ejg','Var193_2Knk1KF','Var193_LrdZy8QqgUfkVShG',
                         'Var193_RO12','Var195_CiJDdr4TQ0rGERIS','Var197_0Xwj','Var197_19FS','Var197_7wJ6',
                         'Var197__8YK','Var198_3ZjbFkF','Var198_B5nJHBC','Var198_bV9grxB','Var198_fhk21Ss',
                         'Var198_itE48fK','Var198_oWeheVi','Var198_pKHsKl9','Var199_8nstQ62E0C',
                         'Var199_EXpWEXgaYpMJ7','Var199_IMbnnzlLVO','Var199_LH0BJkjM1L','Var199_TbQyCeZ21flwy',
                         'Var199_ZIX6Y9cVy5','Var199_ohKlg1a','Var199_q2tfFQq','Var199_qj0HWjvF92VDZUoi',
                         'Var199_yEFIHeU','Var200_Missing','Var202_0iyF','Var202_3taI','Var202_5hez','Var202_5xto',
                         'Var202_6x6x','Var202_C5OU','Var202_LvoD','Var202_OFh8','Var202_Pkzj','Var202_TVqY',
                         'Var202_VDOY','Var202_YMPo','Var202_vVrQ','Var202_xDPJ','Var203_9_Y1','Var204_TjV7',
                         'Var204__r21','Var205_VpdQ','Var205_sJzTlal','Var206_IYzP','Var206_hAFG','Var206_haYg',
                         'Var206_y6dw','Var206_zm5i','Var207_7M47J5GA0pTYIFxg5uy','Var207_DHn_WUyBhW_whjA88g9bvA64_',
                         'Var207_me75fM6ugJ','Var210_g5HH','Var210_uKAI','Var212_9fipLB8rlL','Var212_CrNX',
                         'Var212_NhsEn4L','Var212_WsRVNrF85oPU_','Var212_XfqtO3UdzaXh_','Var214_Missing',
                         'Var216_7Ww_Xqy','Var216_7WwuNea','Var216_7WwwAA_','Var216_NGZXJ7z','Var216_NGZrD3j',
                         'Var216_XTbPUYD','Var216__JdcICD','Var216_kq0FDQL','Var217_2Rt3','Var217_4pYq','Var217_7Rg_',
                         'Var217_9d9v','Var217_Missing','Var217_VA4Q','Var217_Xzba','Var217__SNV','Var217_a6y6',
                         'Var217_jtMm','Var217_rq45','Var217_x9yd','Var218_Missing','Var218_UYBR','Var218_cJvF',
                         'Var220_3Prm94s','Var220_4UxGlow','Var220_6JW488r','Var220_8Qv6hzI','Var220__vd0Q3K',
                         'Var220_aAJYH7D','Var220_eTk1UQz','Var221_d0EEeJi','Var221_oslk','Var221_zCkv',
                         'Var222_2O2Rt2K','Var222_AQ0QowC','Var222_VyMCL5c','Var222_b6TozkN','Var222_catzS2D',
                         'Var222_nDh2fT4','Var222_sd7w991','Var225_ELof','Var225_Missing','Var226_FSa2',
                         'Var227_6fzt','Var227_RAYp','Var227_ZI9m','Var228_55YFVY9','Var228_F2FyR07IdsN7I',
                         'Var228_NoEd','Var228_TCU50_Yjmm6GIBZ0lL_','Var228_ib5G6X1eUxUn6','Var228_iyHGyLCEkQ',
                         'Var229_Missing','Var229_mj86']

## Modelisation Output for Kaggle with the split

In [58]:
# Best feature selection 
selectedFeatures = variables_132

In [967]:
kaggle_output = test[common_col][selectedFeatures] 

In [968]:
len(selectedFeatures)

132

In [969]:
kaggle_output.head()

,Var126_nan,Var7,Var13,Var65,Var73,Var81,Var113,Var126,Var134,Var140,...,Var227_ZI9m,Var228_55YFVY9,Var228_F2FyR07IdsN7I,Var228_NoEd,Var228_TCU50_Yjmm6GIBZ0lL_,Var228_ib5G6X1eUxUn6,Var228_iyHGyLCEkQ,Var229_Missing,Var229_mj86,"Var147_(3.429, 4.571]"
0,0,0.0,0.0,9.0,18,214919.7,-10550.16,-8.000000,428230.0,0.0,...,0,0,1,0,0,0,0,1,0,0
1,1,7.0,44.0,9.0,132,18880.8,-542628.00,-0.498959,207154.0,15.0,...,0,1,0,0,0,0,0,0,1,0
2,0,0.0,0.0,9.0,12,202726.2,-251792.00,-12.000000,691200.0,0.0,...,0,0,1,0,0,0,0,1,0,0
3,1,7.0,72.0,9.0,36,22482.0,-51358.40,-0.498959,830568.0,1120.0,...,0,0,1,0,0,0,0,1,0,0
4,1,7.0,160.0,9.0,58,229352.1,112143.60,-0.498959,2520300.0,35.0,...,0,0,1,0,0,0,0,1,0,0


In [970]:
# Predict using the model based on the splitted data 
kaggle_output["churn"] = bagging2.predict_proba(kaggle_output)[:,1]

In [971]:
kaggle_output.head()

,Var126_nan,Var7,Var13,Var65,Var73,Var81,Var113,Var126,Var134,Var140,...,Var228_55YFVY9,Var228_F2FyR07IdsN7I,Var228_NoEd,Var228_TCU50_Yjmm6GIBZ0lL_,Var228_ib5G6X1eUxUn6,Var228_iyHGyLCEkQ,Var229_Missing,Var229_mj86,"Var147_(3.429, 4.571]",churn
0,0,0.0,0.0,9.0,18,214919.7,-10550.16,-8.000000,428230.0,0.0,...,0,1,0,0,0,0,1,0,0,0.160888
1,1,7.0,44.0,9.0,132,18880.8,-542628.00,-0.498959,207154.0,15.0,...,1,0,0,0,0,0,0,1,0,0.108164
2,0,0.0,0.0,9.0,12,202726.2,-251792.00,-12.000000,691200.0,0.0,...,0,1,0,0,0,0,1,0,0,0.157571
3,1,7.0,72.0,9.0,36,22482.0,-51358.40,-0.498959,830568.0,1120.0,...,0,1,0,0,0,0,1,0,0,0.128318
4,1,7.0,160.0,9.0,58,229352.1,112143.60,-0.498959,2520300.0,35.0,...,0,1,0,0,0,0,1,0,0,0.105571


In [972]:
# Prepare the data for Kaggle format
kaggle_output = pd.DataFrame(test["cust_id"]).join(kaggle_output["churn"])

In [973]:
#kaggle_output.to_csv('output_Kaggle_Bagging_model_132_features.csv',index=False) 

## Modelisation Output for Kaggle with all the data (no split)

In [845]:
test[common_col][selectedFeatures].head(1)

,Var126_nan,Var7,Var13,Var65,Var73,Var81,Var113,Var126,Var134,Var140,...,Var227_ZI9m,Var228_55YFVY9,Var228_F2FyR07IdsN7I,Var228_NoEd,Var228_TCU50_Yjmm6GIBZ0lL_,Var228_ib5G6X1eUxUn6,Var228_iyHGyLCEkQ,Var229_Missing,Var229_mj86,"Var147_(3.429, 4.571]"
0,0,0.0,0.0,9.0,18,214919.7,-10550.16,-8.0,428230.0,0.0,...,0,0,1,0,0,0,0,1,0,0


In [853]:
#choose the best model
model1 = baggingGS
# Fit data 
model1.fit(db_numeric[common_col][selectedFeatures], train["churn"])
print("Model Fit")

#Predict
churn_result = pd.DataFrame(test["cust_id"])
churn_result["churn"] = model1.predict_proba(test[common_col][selectedFeatures])[:,1]
print("TestSet predicted")

#CSV creation
#churn_result.to_csv('kaggleEND6-ALLDATA-BAGwithGBwithGridSearch-132.csv',index=False)
print("CSV file created")

Model Fit
TestSet predicted
CSV file created


## Feature Selection other methods

In [ ]:
stop

#### SequentialFeatureSelector

In [96]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs1 = SFS(boostedTree,
           k_features=20,
           forward=True, 
           floating=False,
           scoring='roc_auc',
           cv=0)

sfs1 = sfs1.fit(db_numeric[selectedFeatures], db_numeric[target])

f300 = sfs1.k_feature_names_
print(sfs1.k_score_)
f300

0.8326969580705114


('Var6',
 'Var13',
 'Var73',
 'Var81',
 'Var125',
 'Var126',
 'Var193_2Knk1KF',
 'Var193_LrdZy8QqgUfkVShG',
 'Var194_Missing',
 'Var194_SEuy',
 'Var195_CiJDdr4TQ0rGERIS',
 'Var195_taul',
 'Var198_kjvBmIU',
 'Var199_r83_sZi',
 'Var201_Missing',
 'Var206_zm5i',
 'Var207_me75fM6ugJ',
 'Var217_RRN_',
 'Var218_UYBR',
 'Var218_cJvF')

#### SelectFrom model

In [ ]:
from sklearn.feature_selection import SelectFromModel
from time import time

threshold = np.sort(importance)[-3] + 0.01

tic = time()
sfm = SelectFromModel(lasso, threshold=threshold).fit(trainingSet[trainingSet_features],trainingSet[target])
toc = time()
print("Features selected by SelectFromModel: ")
      f"{feature_names[sfm.get_support()]}")
print(f"Done in {toc - tic:.3f}s")

In [ ]:
print("Features selected by SelectFromModel: "
      f"{feature_names[sfm.get_support()]}")
print(f"Done in {toc - tic:.3f}s")

#### SelectBest 

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
X_new = SelectKBest(chi2, k=2).fit_transform(trainingSet[trainingSet_features],trainingSet[target])